The point of this notebook is to look at traces of different length and
experiment with different methods of diagnosing them, with the ultimate aim of seeing that they converge properly
Perhaps we will move on to evaluating the traces

In [10]:
import pymc_model as pm
import pymc
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import latin_hypercube as lh
from pymc.Matplot import plot
from pandas.plotting import autocorrelation_plot
import pygemfxns_ensemble_sampling as es
import v2_run_calibration_4Tushar as v2

Upload different databases, each with its own sample length

In [2]:
s1 = pymc.database.pickle.load('test2length1000.pickle')
s3 = pymc.database.pickle.load('test2length3000.pickle')
s5 = pymc.database.pickle.load('test2length5000.pickle')
s10 = pymc.database.pickle.load('test2length10000.pickle')
s30 = pymc.database.pickle.load('test2length30000.pickle')
sample_dict = {'1000':s1, '3000':s3, '5000':s5, '10000':s10, '30000':s30}

Start by comaring the means and standard deviations of each database

In [3]:
for key, value in sample_dict.items():
    mean = np.mean(value.trace('mass_bal')[:])
    std = np.std(value.trace('mass_bal')[:])
    print(key + ' samples:', mean, std)

1000 samples: -0.562115960506 0.225390596859
3000 samples: -0.541571378174 0.242510953646
5000 samples: -0.54631709463 0.238941163444
10000 samples: -0.531816067932 0.23496921524
30000 samples: -0.539429164038 0.236528453722


Use the built in pymc function to look at summary statistics for each variable. 
I was told that MC error should be less than 1% of mean, which is not actually 
a condition that is met even in the 10000 sample

Now we look at a sampling of parameter sets over the space of the Markov chain; ie, we look at the mass balance value that each parameter set leads to, we sort the mass balance values, section them into chunks of equal probability, and randomly sample from each chunk. This gives us a 'stratified' sampling that covers the space of mass balance values but does so with the same mean and standard deviation that the entire chain does. 

In [5]:
for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = lh.sample2(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)

1000 samples, mean: -0.5621967638024394 std: 0.2253690063770066
3000 samples, mean: -0.5417365395885467 std: 0.24237730227976598
5000 samples, mean: -0.546621335550779 std: 0.23707697848716452
10000 samples, mean: -0.5319464494448797 std: 0.2345919100919115
30000 samples, mean: -0.5395731970783786 std: 0.2362750031162424


Now we look at the traces of each variable (thinned down) in the samples of different samples to see whether they have achieved a 'grassy' state, which is what we would expect as the chain converges

In [6]:
for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = es.stratified_sample(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)

1000 samples, mean: -0.5620196272290008 std: 0.2250410309996986
3000 samples, mean: -0.5416449308147085 std: 0.24179359290179978
5000 samples, mean: -0.5466031222172559 std: 0.23908422595812695
10000 samples, mean: -0.5319407572680862 std: 0.23517857234804568
30000 samples, mean: -0.5388537019683503 std: 0.23694704788211726


In [7]:
s1 = pymc.database.pickle.load('test3length1000.pickle')
s3 = pymc.database.pickle.load('test3length3000.pickle')
s5 = pymc.database.pickle.load('test3length500.pickle')
sample_dict = {'500':s5, '1000':s1, '3000':s3}

In [13]:
v2.get_glacier_data(10075)

(0.084000000000000005, 0.23999999999999999, 0)

In [8]:
for key, value in sample_dict.items():
    mean = np.mean(value.trace('mass_bal')[:])
    std = np.std(value.trace('mass_bal')[:])
    print(key + ' samples:', mean, std)

500 samples: 0.251974064303 0.267120314569
1000 samples: 0.277096385146 0.253354831738
3000 samples: 0.267432746393 0.263555535698


In [9]:
for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = es.stratified_sample(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)

500 samples, mean: 0.25321112617784974 std: 0.27252121480931085
1000 samples, mean: 0.27733754910745473 std: 0.25231753945451824
3000 samples, mean: 0.2675192225749556 std: 0.26334070437919954


In [14]:
v2.get_glacier_data(10060)

(-0.040999999999999995, 0.23899999999999999, 3)

In [16]:
s1 = pymc.database.pickle.load('test4length1000.pickle')
#s3 = pymc.database.pickle.load('test4length3000.pickle')
s5 = pymc.database.pickle.load('test4length500.pickle')
sample_dict = {'500':s5, '1000':s1} #, '3000':s3}

In [17]:
for key, value in sample_dict.items():
    mean = np.mean(value.trace('mass_bal')[:])
    std = np.std(value.trace('mass_bal')[:])
    print(key + ' samples:', mean, std)

500 samples: 0.100868109493 0.295260450306
1000 samples: 0.068644963584 0.246338442658


In [18]:
for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = es.stratified_sample(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)

500 samples, mean: 0.10237250948342419 std: 0.3031835812660306
1000 samples, mean: 0.0686063534031408 std: 0.2460814988288975
